In [1]:
import pandas as pd

In [2]:
geo = pd.read_excel('题目一：地名数据库.xlsx', sheet_name='霍普金斯地名')
geo

,Column1,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key
0,NaN,4,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.93911,67.709953,Afghanistan
1,NaN,8,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.15330,20.168300,Albania
2,NaN,12,DZ,DZA,12.0,NaN,NaN,NaN,Algeria,28.03390,1.659600,Algeria
3,NaN,16,AS,ASM,16.0,60.0,NaN,American Samoa,US,-14.27100,-170.132000,"American Samoa, US"
4,NaN,20,AD,AND,20.0,NaN,NaN,NaN,Andorra,42.50630,1.521800,Andorra
...,...,...,...,...,...,...,...,...,...,...,...,...
3563,NaN,84090053,US,USA,840.0,90053.0,Unassigned,Washington,US,NaN,NaN,"Unassigned, Washington, US"
3564,NaN,84090054,US,USA,840.0,90054.0,Unassigned,West Virginia,US,NaN,NaN,"Unassigned, West Virginia, US"
3565,NaN,84090055,US,USA,840.0,90055.0,Unassigned,Wisconsin,US,NaN,NaN,"Unassigned, Wisconsin, US"
3566,NaN,84090056,US,USA,840.0,90056.0,Unassigned,Wyoming,US,NaN,NaN,"Unassigned, Wyoming, US"


In [50]:
iso2 = pd.read_excel('题目一：地名数据库.xlsx', sheet_name='ISO3166-2')
iso2

,国家,Code,Subdivision name,Subdivision name (cn),Subdivision name (en),Subdivision Category,Parent subdivision,ISO 3166-2相应代码
0,Andorra,AD-02,Canillo,NaN,NaN,parish,NaN,ISO 3166-2:AD-02
1,Andorra,AD-03,Encamp,NaN,NaN,parish,NaN,ISO 3166-2:AD-03
2,Andorra,AD-04,La Massana,NaN,NaN,parish,NaN,ISO 3166-2:AD-04
3,Andorra,AD-05,Ordino,NaN,NaN,parish,NaN,ISO 3166-2:AD-05
4,Andorra,AD-06,Sant Julià de Lòria,NaN,NaN,parish,NaN,ISO 3166-2:AD-06
...,...,...,...,...,...,...,...,...
6235,Zimbawe,ZW-MI,Midlands,NaN,NaN,province,NaN,ISO 3166-2:ZW-MI
6236,Zimbawe,ZW-MN,Matabeleland North,NaN,NaN,province,NaN,ISO 3166-2:ZW-MN
6237,Zimbawe,ZW-MS,Matabeleland South,NaN,NaN,province,NaN,ISO 3166-2:ZW-MS
6238,Zimbawe,ZW-MV,Masvingo,NaN,NaN,province,NaN,ISO 3166-2:ZW-MV


In [115]:
alpha2 = []
for i in iso2.index:
    alpha2.append(iso2['Code'].iloc[i][:2])
    
iso2['a2'] = alpha2

In [120]:
common = set(iso2['a2']) & set(geo['iso2'])

In [232]:
from lxml import etree
import urllib.request
import urllib.parse

In [138]:
def lookforen(a2):
    url = 'https://en.wikipedia.org/wiki/ISO_3166-2:' + a2

    headers = { 
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36' 
    }

    req = urllib.request.Request(url=url, headers=headers, method='GET')

    response = urllib.request.urlopen(req)

    text = response.read().decode('utf-8')
    html = etree.HTML(text)
    obj_list = html.xpath('//div[@class="mw-parser-output"]/*')
    
    for tmp in obj_list:
        if tmp.tag == 'table':
            table = tmp.xpath('.//text()')
            break
        
    raw = [table[i] for i in range(len(table)) if table[i] not in ['\n',' ','\xa0']]
    
    for i in iso2[iso2['a2'] == a2]['Code'].index:
        for j in range(len(raw)):
            if iso2['Code'].iloc[i] == raw[j]:
                iso2['Subdivision name (en)'].iloc[i] = raw[j+1]

In [229]:
def is_Chinese(word):
    for ch in word:
        if '\u4e00' <= ch <= '\u9fff':
            return True
    return False

def lookforzh(a2):
    url = 'https://zh.wikipedia.org/wiki/ISO_3166-2:' + a2

    headers = { 
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36' 
    }

    req = urllib.request.Request(url=url, headers=headers, method='GET')
    
    try:
        response = urllib.request.urlopen(req)
        text = response.read().decode('utf-8')
        html = etree.HTML(text)
        obj_list = html.xpath('//div[@class="mw-parser-output"]/*')

        for tmp in obj_list[1:]:
            if tmp.tag == 'table':
                table = tmp.xpath('.//text()')
                break

        raw = [table[i] for i in range(len(table)) if table[i] not in ['\n',' ','\xa0']]

        for i in iso2[iso2['a2'] == a2]['Code'].index:
            for j in range(len(raw)):
                if iso2['Code'].iloc[i] == raw[j]:
                    if is_Chinese(raw[j+1]):
                        iso2['Subdivision name (cn)'].iloc[i] = raw[j+1]
                    elif j+2 < len(raw):
                        iso2['Subdivision name (cn)'].iloc[i] = raw[j+2]
    except urllib.request.HTTPError: # 404, 500, etc..
        pass


In [141]:
for a2 in common:
    lookforen(a2)

In [230]:
for a2 in common:
    lookforzh(a2)

In [231]:
iso2.to_excel('data/iso3166_2_final.xlsx')